# Initialize a GPT Model

In [32]:
from transformers import AutoConfig, AutoModelForCausalLM, GPT2Tokenizer
from transformers import GPT2Tokenizer

vocab_size = 13
sequence_length = 4
result_length = 2
context_length = sequence_length + result_length
config = AutoConfig.from_pretrained("gpt2", vocab_size=vocab_size, n_ctx=context_length, n_head=4, n_layer=2)
model = AutoModelForCausalLM.from_config(config)

The model will have the same architecture as GPT 2 but with a few modifications for making it smaller. The main changes are the size of the vocabulary that it is 13 because it will only handle numbers plus the padding token, the "+", and "=". The context window will only support 6 tokens since we are only interested in performing the addition of two single digits.

In [29]:
def model_size(model):
    return sum(t.numel() for t in model.parameters())

print(f'Model size: {model_size(model)/1000**2:.1f}M parameters')

Model size: 15.0M parameters


This model have 15 million weights instead of the 111 million parameters of the "gpt2" default config.

In [30]:
model_ckpt = 'addition_model'

In [31]:
model.save_pretrained("models/" + model_ckpt, push_to_hub=True)

pytorch_model.bin: 100%|██████████| 59.9M/59.9M [00:23<00:00, 2.59MB/s]


# Custom tokenizer to encode numbers and the padding token

In [103]:
class NumberTokenizer:
  def __init__(self, numbers_qty=10):
    vocab = ['+', '=', '-1', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    self.numbers_qty = numbers_qty
    self.pad_token = '-1'
    self.encoder = {str(v):i for i,v in enumerate(vocab)}
    self.decoder = {i:str(v) for i,v in enumerate(vocab)}
    self.pad_token_id = self.encoder[self.pad_token]

  def decode(self, token_ids):
    return ' '.join(self.decoder[t] for t in token_ids)

  def __call__(self, text):
    return [self.encoder[t] for t in text.split()]

### Example of the tokenization for the model

In [104]:
tokenizer = NumberTokenizer(vocab_size)
tokenizer("1 + 1 = 2")

[4, 0, 4, 1, 5]

In [105]:
tokenizer.decoder

{0: '+',
 1: '=',
 2: '-1',
 3: '0',
 4: '1',
 5: '2',
 6: '3',
 7: '4',
 8: '5',
 9: '6',
 10: '7',
 11: '8',
 12: '9'}

# Built dataset with unsorted and sorted sequences of numbers

In [144]:
import numpy as np
import torch
from torch.utils.data import Dataset

class AdditionDataset(Dataset):
    """ 
    Dataset with example of the form:
    Input: "2 + 3 = 0" where the first 4 characters represent the initial input sequence
    Output: "+ 3 = 0 5" where the last ditis represents the result of the addition and the first 3 digits are ignored during training
    The result is a dataset of tokenized sequences of numbers
    """

    def __init__(self, split, length=6):
        assert split in {'train', 'test'}
        self.split = split
        self.length = length
    
    def __len__(self):
        return 1000000 # 1M examples per split
    

    def __getitem__(self, idx):
        # The dataset is generated on the fly

        available_numbers = [int(n) for n in tokenizer.decoder.values() if n != tokenizer.pad_token and str(n).isnumeric()]
        # generate some random integers
        inp = torch.tensor(np.random.choice(available_numbers, size=result_length))
        # solve the task
        sol = torch.tensor([int(i) for i in str(inp.sum().item())])
        sol = torch.nn.functional.pad(sol, (1 if sol.size()[0] == 1 else 0,0), 'constant', 0)

        
        # concatenate the problem specification and the solution
        cat = torch.cat((inp, sol), dim=0)

        # the inputs to the transformer will be the offset sequence
        x = cat[:-1].clone()
        y = cat[1:].clone()
        # we only want to predict at output locations, mask out the loss at the input locations
        y[:1] = int(tokenizer.pad_token)

        # Convert the tensors to the input expected by the model
        x = str(x[0].item()) + ' + ' + str(x[1].item()) + ' = ' + str(x[2].item())
        y = '-1 ' + str(y[0].item()) + ' -1 ' + str(y[1].item()) + ' ' + str(y[2].item())
        # tokenize the input and targets
        tokenized_input = tokenizer(x)
        tokenized_output = tokenizer(y)
        return torch.tensor(tokenized_input), torch.tensor(tokenized_output)

### Create the training and testing datasets

In [149]:
train_dataset = AdditionDataset('train', length=sequence_length)
test_dataset = AdditionDataset('test', length=sequence_length)

In [150]:
x, y = train_dataset[0]
print(tokenizer.decode(x.numpy()))
print(tokenizer.decode(y.numpy()))

5 + 7 = 1
-1 -1 -1 1 2


# Generating solutions

In [151]:
def generate_solution(input, solution_length=6, model=model):
  model.eval()
  input = torch.tensor(tokenizer(input))
  input = input.to(accelerator.device)
  solution = []
  for i in range(solution_length):
    output = model(input)
    predicted = output.logits[-1].argmax()
    input = torch.cat((input, predicted.unsqueeze(0)), dim=0)
    solution.append(predicted.cpu().item())
  return tokenizer.decode(solution)

# Training loop

In [152]:
import torch
import torch.nn.functional as F
from accelerate import Accelerator

accelerator = Accelerator()

batch_size = 100
num_epochs = 1

optimizer = torch.optim.Adam(model.parameters())
dataset = train_dataset
data = torch.utils.data.DataLoader(dataset, shuffle=True, batch_size=batch_size)

model, optimizer, data = accelerator.prepare(model, optimizer, data)

model.train()
for epoch in range(num_epochs):
  for source, targets in data:
    optimizer.zero_grad()
    loss = F.cross_entropy(model(source).logits.flatten(end_dim=1), targets.flatten(end_dim=1), ignore_index=tokenizer.pad_token_id)
    accelerator.backward(loss)
    optimizer.step()
    loss = F.cross_entropy(model(source).logits.flatten(end_dim=1), targets.flatten(end_dim=1), ignore_index=tokenizer.pad_token_id)
  print(f'Epoch: {epoch+1}/{num_epochs} loss: {loss.item()}')

Epoch: 1/1 loss: 0.18201486766338348


# Evaluating the model

In [156]:
def evaluate_model(num_samples=1000, log=False):
  correct = 0
  for i in range(num_samples):
    input, target = test_dataset[i]
    input = input.cpu().numpy()
    target = target.cpu().numpy()
    input = tokenizer.decode(input[:sequence_length])
    target = tokenizer.decode(target[sequence_length-1:])
    predicted = generate_solution(input, solution_length=result_length, model=model)
    if target == predicted:
      correct += 1
      if log:
        print(f'CORRECT  Input: {input} Target: {target} Predicted: {predicted}')
    else:
      if log:
        print(f'Input: {input} Target: {target} Predicted: {predicted}')

  print(f'Accuracy: {correct/num_samples}')

In [157]:
evaluate_model(num_samples=1000, log=False)

Accuracy: 0.946


In [158]:
evaluate_model(num_samples=10, log=True)

CORRECT  Input: 6 + 3 = Target: 0 9 Predicted: 0 9
CORRECT  Input: 0 + 4 = Target: 0 4 Predicted: 0 4
CORRECT  Input: 4 + 4 = Target: 0 8 Predicted: 0 8
CORRECT  Input: 0 + 4 = Target: 0 4 Predicted: 0 4
CORRECT  Input: 7 + 6 = Target: 1 3 Predicted: 1 3
CORRECT  Input: 4 + 7 = Target: 1 1 Predicted: 1 1
CORRECT  Input: 9 + 8 = Target: 1 7 Predicted: 1 7
CORRECT  Input: 5 + 5 = Target: 1 0 Predicted: 1 0
CORRECT  Input: 1 + 3 = Target: 0 4 Predicted: 0 4
CORRECT  Input: 9 + 9 = Target: 1 8 Predicted: 1 8
Accuracy: 1.0


In [159]:
model.push_to_hub("models/" + model_ckpt)

pytorch_model.bin: 100%|██████████| 59.9M/59.9M [00:37<00:00, 1.60MB/s]


CommitInfo(commit_url='https://huggingface.co/Manuel2011/addition_model/commit/d3600b85f53ef59f02dd579bc15326a618d62c39', commit_message='Upload model', commit_description='', oid='d3600b85f53ef59f02dd579bc15326a618d62c39', pr_url=None, pr_revision=None, pr_num=None)